# Продвинутый Python, лекция 8

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Продолжаем парсить данные, только поумнее

В прошлый раз мы с вами парсили странички с помощью BS и в целом были этим довольны. Давайте немного усложним задачу: хотим целую систему по парсингу данных!

Для этого есть прекрасная бибилотека [Scrapy](https://scrapy.org/)!

Суть простая:

1. Создаем своего павука, который будет ходить по необходимым сайтам и забирать оттуда результаты

2. Результат мы передаем в парсер, чтобы вытащить необходимые нам данные

3. Эти данные мы сохраняем в файлики

4. ...

5. PROFIT

![](https://pustunchik.ua/uploads/school/cache/old/interesting/animals/Chogo-ty-ne-znav-pro-tvaryn/pauki_foto_08.jpg)

Сегодня мы будем с вами парсить Amazon (доставать оттуда товары и их характеристики)

In [ ]:
!pip install scrapy

Поскольку паук - это уже целый проект, то и соответственно, одни скриптом здесь не обойтись. Поэтому при создании паука через Scrapy мы получаем сразу целую папку

In [2]:
!scrapy startproject spider_amazon; cd spider_amazon; ls -la

New Scrapy project 'scrapy_amazon', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/scrapy_amazon

You can start your first spider with:
    cd scrapy_amazon
    scrapy genspider example example.com
total 16
drwxr-xr-x 3 root root 4096 Oct 29 14:27 .
drwxr-xr-x 1 root root 4096 Oct 29 14:27 ..
drwxr-xr-x 3 root root 4096 Oct 29 14:27 scrapy_amazon
-rw-r--r-- 1 root root  269 Oct 29 14:27 scrapy.cfg


In [5]:
!cd spider_amazon/spider_amazon; ls -la

total 28
drwxr-xr-x 3 root root 4096 Oct 29 14:27 .
drwxr-xr-x 3 root root 4096 Oct 29 14:27 ..
-rw-r--r-- 1 root root    0 Oct 29 14:20 __init__.py
-rw-r--r-- 1 root root  268 Oct 29 14:27 items.py
-rw-r--r-- 1 root root 3660 Oct 29 14:27 middlewares.py
-rw-r--r-- 1 root root  366 Oct 29 14:27 pipelines.py
-rw-r--r-- 1 root root 3325 Oct 29 14:27 settings.py
drwxr-xr-x 2 root root 4096 Oct 29 14:20 spiders


Что мы здесь видим? Кучу различных питоновских скриптов, которые надо заполнить, чтобы привести нашего паука в действие!

Давайте разбираться с каждым отдельно и постигать, как же все это построить. Начнем с наиболее простого

### Items

In [6]:
!cat spider_amazon/spider_amazon/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ScrapyAmazonItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


Что такое Item? По существу, это опять-таки dataclass, внутри которого будут храниться необходимые вам поля. Зачем это нужно? Чтобы получать информацию в виде объектов, которые вы можете передать далее и доставать из них нужную информацию

Облегчаем себе жизнь, доставая лишь то, что нужно в красивой обертке. Помимо встреоенного scrapy.Item можно также сюда делать непосредственно dataclass и словари. Самое главное - что это то, что вы по итогу получаете

Давайте на примере:

In [7]:
import scrapy

class Product(scrapy.Item):
    name = scrapy.Field() # Что такое field? Алиас к словарю (то есть это словарь)
    price = scrapy.Field()
    stock = scrapy.Field()
    tags = scrapy.Field()

In [8]:
product = Product(name='Desktop PC', price=1000)
product

{'name': 'Desktop PC', 'price': 1000}

Получили словарь, ничего интересного. Давайте заполним теперь для нашего Амазон-парсера items

Какие аттрибуты хотим? Базово скажем:

* Название (name)

* Цена без скидки (original_price)

* Категория (category)

* Цена со скидкой (sale_price)

* Можно ли купить (availability) 

Заносим в ```items.py```

In [79]:
import scrapy

class SpiderAmazonItem(scrapy.Item):
    # define the fields for your item here like:
    product_name = scrapy.Field()
    product_sale_price = scrapy.Field()
    product_category = scrapy.Field()
    product_original_price = scrapy.Field()
    product_availability = scrapy.Field()

Отлично, настроили коробку для презентации наших результатов, двинемся дальше

### Spiders

Spiders - это основа нашего проекта, паучок, который будет ходить по сайту и забирать данные, чтобы потом передавать их на обработку. Если сейчас посмотреть внутри папки Spiders, то там есть только init, в котором ничего нет. Давайте создадим дефолтного паука и будем его оснащать

In [ ]:
!scrapy genspider AmazonProductSpider amazon.com

Отлично, создался некоторый дефолтный паук (внутри папки spiders файл AmazonProductSpider.py)

```
import scrapy


class AmazonproductspiderSpider(scrapy.Spider):
    name = 'AmazonProductSpider'
    allowed_domains = ['amazon.com']
    start_urls = ['http://amazon.com/']

    def parse(self, response):
        pass

```

Что тут есть изначально?

* name - название нашего паука (к которому мы сможем обращаться из командной строки)

* allowed_domains - список доменов, внутри которых наш паук может передвигаться (по дефолту ограничиваем, что только по amazon, на остальные сайты не ходим)

* start_urls - с каких ссылок мы начинаем ходить

После того, как мы зашли на страничку, далее происходит parse, в качестве аргумента которого передается response, то есть контент, который мы получили от GET-запроса. Для того, чтобы написать parse, нам надо обратиться уже к сайту Amazon

Возьмем несколько ссылок по флэшкам, например:

* https://www.amazon.com/dp/B00JES3MO0

* https://www.amazon.com/dp/B08NCC24HV

* https://www.amazon.com/dp/B09LCDF2VD

* https://www.amazon.com/dp/B0B3D8Z7T8

In [ ]:
from bs4 import BeautifulSoup

s = ""
with open("Amazon.txt", 'r') as f:
    s = f.read()
soup = BeautifulSoup(s, 'html.parser') # указываем парсер
print(soup.prettify()) # выглядит уже более структурно

Справедливости ради, выглядит ужасно, иногда проще просто по самой странице искать, что просто по HTML-коду. Давайте разыскивать все, что нам надо

Что видим?

* Название - ```id=productTitle```

* Категория - ```<a class="a-link-normal a-color-tertiary" ... > </a> ```

* Цена без скидки -  ```<span class="... a-price a-text-price'">```

* Цена со скидкой - ```<span class="... priceToPay">```

* Можно ли купить - ```<div id="availability" ... > ```

In [13]:
soup.find_all(id='productTitle')[0].text.strip()

'Enfain 16GB USB 2.0 Flash Memory Stick Drive Swivel Thumb Drives Bulk 10 Pack Black, Portable Data Storage for Universal Purposes at Home & The Office'

In [34]:
soup.find_all(class_ = 'a-link-normal a-color-tertiary')[0].text.strip() + '/' + \
soup.find_all(class_ = 'a-link-normal a-color-tertiary')[1].text.strip() + '/' + \
soup.find_all(class_ = 'a-link-normal a-color-tertiary')[2].text.strip() + '/' + \
soup.find_all(class_ = 'a-link-normal a-color-tertiary')[3].text.strip()

'Electronics/Computers & Accessories/Data Storage/USB Flash Drives'

In [70]:
soup.find_all(class_ = 'a-price a-text-price')

[<span class="a-price a-text-price" data-a-color="secondary" data-a-size="s" data-a-strike="true"><span class="a-offscreen">$30.80</span><span aria-hidden="true">$30.80</span></span>]

In [25]:
soup.find_all(class_ = 'a-price a-text-price')[0].span.text

'$30.80'

In [67]:
soup.find_all(class_ = "priceToPay")

[<span class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay" data-a-color="base" data-a-size="xl"><span class="a-offscreen">$24.80</span><span aria-hidden="true"><span class="a-price-symbol">$</span><span class="a-price-whole">24<span class="a-price-decimal">.</span></span><span class="a-price-fraction">80</span></span></span>]

In [29]:
soup.find_all(class_ = "priceToPay")[0].span.text

'$24.80'

In [31]:
soup.find_all(id='availability')[0].span.text.strip()

'In Stock.'

Ура, все нашли, но выглядит не очень, но все равно надо как-то записать, да и еще через scrapy. Но чтобы облегчить нашу работу, погорим про такую вещь, как XPath (XML Path Language)

### Xpath

Как мы с вами на семинаре говорили, что XML, по своему суещству, это та же HTML-разметка, только с меньшими условностями (можно заводить вообще любые тэги)

А это значит, что HTML-документ можно рассматривать как пример XML-документа и его парсить!



In [82]:
from lxml import etree

dom = etree.HTML(str(soup))

XPath - это язык для путей внутри XML. То есть можно в одну строчку написать, как пройти к необходимому полю, частично мы такое уже видели в BS (через, например, a/span - вложенность span внутри a)

Можно считать, что XPath - это определенные регулярки, только для файлов. Давайте разберемся в базовом синтаксисе [XPath](http://www.k-press.ru/cs/2001/2/XPath/XPath4.asp):

* /  - спуск по иерархии (прямо как с директориями-файлами)

* // - рекурсивный спуск (вытягиваем все внутри на любой глубине)

* . - где мы находимся сейчас

* @ - атрибут (например, если мы ищем все span с class="abc", то это будет @class="abc"

* \* - выбираем все (зачем нужно: допустим, что мы хотим найти все a внутри span на 1 уровень ниже, то есть span/что угодно/a, тогда запись будет как span/*/a)

* [] - фильтр и индексация

* and, or, not - логические операции

* contains - поиск по тому, присутствует ли нужный атрибут (например, contains(@class, "abc") - если есть атрибут class и внутри class есть "abc")

Разберем на примере:

```
//span[@class="a-price a-text-price"]/span[@class="a-offscreen"]/text()
```

1. //span - найди рекурсивно тэг span (вне зависимости от уровня)

2. [@class="a-price a-text-price"] - отфильтруй их по признаку class = "a-price a-text-price"

3. /span[@class="a-offscreen"] - Перейди по отфильтрованным в дочерний span с нужным классом

4. /text() - вытяни текст

In [83]:
dom.xpath('//span[@class="a-price a-text-price"]/span[@class="a-offscreen"]/text()')

['$30.80']

In [90]:
dom.xpath('//span[@class="a-price a-text-price"]/span[1]//text()') # аналогично, только переходим во второй span

['$30.80']

Второй пример:

```
//span[contains(@class,"priceToPay")]/span[@class="a-offscreen"]//text()
```

Здесь разница только в том, что мы используем contains (то есть если в классе есть priceToPay, то забираем). Почему здесь используем contains? Потому что если посмотреть выше, то там не только этот класс присутствует

Давайте запишем:

In [80]:
def parse(response):
    items = SpiderAmazonItem()
    title = response.xpath('//span[@id="productTitle"]/text()')#.extract() - внутри scrapy работает тот же xpath, но только результат надо отдельно вытаскивать
    sale_price = response.xpath('//span[contains(@class,"priceToPay")]/span[@class="a-offscreen"]//text()')#.extract()
    full_price = response.xpath('//span[@class="a-price a-text-price"]/span[@class="a-offscreen"]//text()')#.extract()
    category = response.xpath('//a[@class="a-link-normal a-color-tertiary"]/text()')#.extract()
    availability = response.xpath('//div[@id="availability"]/span//text()')#.extract()
    items['product_name'] = ''.join(title).strip()
    items['product_sale_price'] = ''.join(sale_price).strip()
    items['product_category'] = '/'.join(map(lambda x: x.strip(), category)).strip()
    items['product_availability'] = ''.join(availability).strip()
    items['product_original_price'] = ''.join(full_price).strip()
    yield items

In [81]:
next(parse(dom))

{'product_availability': 'In Stock.',
 'product_category': 'Electronics/Computers & Accessories/Data Storage/USB '
                     'Flash Drives',
 'product_name': 'Enfain 16GB USB 2.0 Flash Memory Stick Drive Swivel Thumb '
                 'Drives Bulk 10 Pack Black, Portable Data Storage for '
                 'Universal Purposes at Home & The Office',
 'product_original_price': '$30.80',
 'product_sale_price': '$24.80'}

Ура, разобрались! Можно двигаться дальше!

### Дособираем паука

In [ ]:
import scrapy

from spider_amazon.items import SpiderAmazonItem

class AmazonproductspiderSpider(scrapy.Spider):
    name = 'AmazonProductSpider'
    allowed_domains = ['amazon.com']
    start_urls = ['https://www.amazon.com/dp/B00JES3MO0', 'https://www.amazon.com/dp/B08NCC24HV',
        'https://www.amazon.com/dp/B09LCDF2VD', 'https://www.amazon.com/dp/B0B3D8Z7T8']

    def parse(self, response):
        items = SpiderAmazonItem()
        title = response.xpath('//span[@id="productTitle"]/text()').extract()
        sale_price = response.xpath('//span[contains(@class,"priceToPay")]/span[@class="a-offscreen"]//text()').extract()
        full_price = response.xpath('//span[@class="a-price a-text-price"]/span[@class="a-offscreen"]//text()').extract()
        category = response.xpath('//a[@class="a-link-normal a-color-tertiary"]/text()').extract()
        availability = response.xpath('//div[@id="availability"]/span//text()').extract()
        items['product_name'] = ''.join(title).strip()
        items['product_sale_price'] = ''.join(sale_price).strip()
        items['product_category'] = '/'.join(map(lambda x: x.strip(), category)).strip()
        items['product_availability'] = ''.join(availability).strip()
        items['product_original_price'] = ''.join(full_price).strip()
        yield items

Отлично, мы с вами написали код, который что-то будет выдавать! Осталось это все дело связать

### Pipelines

Все, что мы получаем после обработки (когда выдаем yield в пауке), мы передаем в pipeline - это обработка полученного результата

```
# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class SpiderAmazonPipeline:
    def process_item(self, item, spider):
        return item
```

Вызывается функция process_item, внутри которой можно обработать получившиеся items из паука (иногда мы хотим что-нибудь отфильтровать, добавить в json etc)

Давайте добавим функционал, чтобы наши результаты можно было добавлять в json файлик

In [ ]:
import json
from itemadapter import ItemAdapter #класс для обработки items, можно проверить, получили ли мы item

class SpiderAmazonPipeline:

    def open_spider(self, spider): # что делать при открытии паука (создаем файлик)
        self.file = open('items.json', 'w')

    def close_spider(self, spider): # что делать при окончании работы паука (закрываем файлик)
        self.file.close()

    def process_item(self, item, spider): #что делать с полученным item
        line = json.dumps(ItemAdapter(item).asdict()) + "\n"
        self.file.write(line)
        return item

### Middlewares

Исходя из названия это некоторый "промежуточный слой". Что он в себе таит?

```
# Define here the models for your spider middleware
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/spider-middleware.html

from scrapy import signals

# useful for handling different item types with a single interface
from itemadapter import is_item, ItemAdapter


class SpiderAmazonSpiderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the spider middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_spider_input(self, response, spider):
        # Called for each response that goes through the spider
        # middleware and into the spider.

        # Should return None or raise an exception.
        return None

    def process_spider_output(self, response, result, spider):
        # Called with the results returned from the Spider, after
        # it has processed the response.

        # Must return an iterable of Request, or item objects.
        for i in result:
            yield i

    def process_spider_exception(self, response, exception, spider):
        # Called when a spider or process_spider_input() method
        # (from other spider middleware) raises an exception.

        # Should return either None or an iterable of Request or item objects.
        pass

    def process_start_requests(self, start_requests, spider):
        # Called with the start requests of the spider, and works
        # similarly to the process_spider_output() method, except
        # that it doesn’t have a response associated.

        # Must return only requests (not items).
        for r in start_requests:
            yield r

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)


class SpiderAmazonDownloaderMiddleware:
    # Not all methods need to be defined. If a method is not defined,
    # scrapy acts as if the downloader middleware does not modify the
    # passed objects.

    @classmethod
    def from_crawler(cls, crawler):
        # This method is used by Scrapy to create your spiders.
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s

    def process_request(self, request, spider):
        # Called for each request that goes through the downloader
        # middleware.

        # Must either:
        # - return None: continue processing this request
        # - or return a Response object
        # - or return a Request object
        # - or raise IgnoreRequest: process_exception() methods of
        #   installed downloader middleware will be called
        return None

    def process_response(self, request, response, spider):
        # Called with the response returned from the downloader.

        # Must either;
        # - return a Response object
        # - return a Request object
        # - or raise IgnoreRequest
        return response

    def process_exception(self, request, exception, spider):
        # Called when a download handler or a process_request()
        # (from other downloader middleware) raises an exception.

        # Must either:
        # - return None: continue processing this exception
        # - return a Response object: stops process_exception() chain
        # - return a Request object: stops process_exception() chain
        pass

    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)

```

По существу это любая обработка поведения паука (при вызове, при выдаче результата etc), здесь таится логирование, непосредственно вызов самого паука etc. Здесь ничего менять не будем

### Settings

Здесь хранятся настройки нашего проекта

```
# Scrapy settings for spider_amazon project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'spider_amazon'

SPIDER_MODULES = ['spider_amazon.spiders']
NEWSPIDER_MODULE = 'spider_amazon.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'spider_amazon (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'spider_amazon.middlewares.SpiderAmazonSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'spider_amazon.middlewares.SpiderAmazonDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'spider_amazon.pipelines.SpiderAmazonPipeline': 300,
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

# Set settings whose default value is deprecated to a future-proof value
REQUEST_FINGERPRINTER_IMPLEMENTATION = '2.7'
TWISTED_REACTOR = 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'
```

Какие настройки надо включить (включаем ItemPipelines)

Что изначально включено?

* ROBOTSTXT_OBEY = True - мы не вредоносные товарищи, поэтому следуем правилам для роботов (иначе получим пермабан для нашего IP)

Что за число 300 у ItemPipelines? Это приоритет (мы можем создать несколько обработчиков, берем по приоритету, числа от 1 до 1000, чем больше - тем приоритетнее, если один не справился, то берет следующий по приоритету)

## Запускаем все это добро!

Упс, что-то не сработало. Давайте смотреть в логи:

503, 503, 503... Amazon спокойно задетектил, что мы роботы и ничего не дал, плак-плак. Надо такие вещи обходить!

Делать будем это с помощью [ScraperAPI](https://www.scraperapi.com/)! 

Что это такое? Давайте вначале в целом поговорим, зачем ботов банить

В сети ходит миллионы (если не больше) различных ботов, которые собирают ту или иную информацию. При этом есть хорошие боты (типа Facebook crawler, который собирает информацию для того, чтобы делать вам лучше рекомендации), а есть различные вредоносные боты, которые пытаются сломать сайт (DDOS-атаки: давайте нагрузим сервер огромным числом запросов, чтобы он упал), хакнуть вам пользователей, использовать в качестве конкуренции (собирать цены на вашем сайте, а потом на их основе вас немного демпинговать) etc

По дефолту сам владелец указывает, что есть хорошо, а что есть плохо. Например, если ему надо собирать статистику etc, то он позволит Facebook Crawler собирать инфу. А всем остальным мы ограничиваем доступ. Если вы посмотрите в логи выполнения нашего crawler, то можете увидеть в первую очередь вот такую вещь:

https://www.amazon.com/robots.txt

Это документ, который говорит, что можно роботу, а что нельзя роботу (и как можно видеть, почти ничего нельзя)


А как можно задетектить бота?

* Слишком много запросов - не похоже на человека

* Слишком быстро уходишь с сайта - роботы это делают за миллисекунды, люди помедленнее будут

* Слишком быстрые сессии - боты быстрее людей

* Иногда можно задетектить по самим ссылкам - попробуйте зайти в условный маркет, тыкнуть на товар и посмотреть на ссылку:

https://market.yandex.ru/product--veb-kamera-logitech-c922-pro-stream/1711878907?skuId=100328764984&sku=100328764984&show-uid=16671261826754605352306000&offerid=OLIPX0hJS-c6i07BZxtEkA&cpc=A1G1H7OJVLmd9_9T1L4EN-tlgcEeW9fAiA2QpnL7U7bOkvGq2sEud-N9sRb5D2VJ36LmEAtHnxbc7itnzvh72MQSmhOyyMgNZYq35-mCHZnkizKWYr-vWBb62Ra9_WpSweQK7HWvl1Advup0V_KVFdX2nw3-e36pyj-ClcKbOeSZayQTaWxcu4OEcsESPBjY&no-pda-redir=1

При этом чтобы попасть на тот же товар, вам достаточно всего лишь:

https://market.yandex.ru/product--veb-kamera-logitech-c922-pro-stream/1711878907

Все остальное - это различные параметры и тэги, которые потом нужны для маркетинговых исселдований трафика etc. Человек, пришедший без ссылок - это тоже подозрительно





Можно очень напрячься и все это учитывать (заходить с разных IP etc), но поскольку web crawling - это достаточно популярная вещь, то есть много сервисов, которые вам позволяют получить прокси, которая самостоятельно будет проходить через капчи и прочие пакости и имитировтаь человеческое поведение (при этом если вы не делаете какую-то гадость)

Вот ScraperAPI - одна из таких утилит.

Нужно зарегистрироваться и вы сразу получаете бесплатную версию (где можно делать до 5000 запросов в месяц по 5 потокам одновременно, больше запросов уже за денежку)

Вы получаете API-ключ, через который вы авторизируетесь, что это ваш запрос, который Scraper прокинет через свои сервера и настройки, в конечном итоге вы получаете доступ!



## Уходим от блокирования роботом

Что нужно сделать? Надо добавить в паучка проход через ScraperAPI!

Добавляем к нашему пауку вне функцию (добавляем функцию для сборки нашего urlа через [urllib.requests](https://docs.python.org/3/library/urllib.request.html) - полезная библиотека, чтобы спарсить url или собрать его как нам надо. Как вообще работают url-ссылки поговорим на семинаре

In [ ]:
import scrapy
from urllib.parse import urlencode
from spider_amazon.items import SpiderAmazonItem

API = 'YOUR_TOKEN'

def get_url(url):
    payload = {'api_key': API, 'url': url}
    proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload) # обращаемся к ScraperAPI, указывая необходимый сайт и API ключ
    return proxy_url

class AmazonproductspiderSpider(scrapy.Spider):
    name = 'AmazonProductSpider'
    allowed_domains = ['amazon.com']
    start_urls = ['https://www.amazon.com/dp/B00JES3MO0', 'https://www.amazon.com/dp/B08NCC24HV',
                  'https://www.amazon.com/dp/B09LCDF2VD', 'https://www.amazon.com/dp/B0B3D8Z7T8']

    def start_requests(self): # придется переписать start_requests - функция, которая работает с GET-запросами
        for url in self.start_urls:
            yield scrapy.Request(url=get_url(url), callback=self.parse)

    # Как эта функция выглядит по дефолту
        # def start_requests(self):
        # if not self.start_urls and hasattr(self, 'start_url'):
        #     raise AttributeError(
        #         "Crawling could not start: 'start_urls' not found "
        #         "or empty (but found 'start_url' attribute instead, "
        #         "did you miss an 's'?)")
        # for url in self.start_urls:
        #     yield Request(url, dont_filter=True)

    def parse(self, response):
        items = SpiderAmazonItem()
        title = response.xpath('//h1[@id="title"]/span/text()').extract()
        sale_price = response.xpath('//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()').extract()
        category = response.xpath('//a[@class="a-link-normal a-color-tertiary"]/text()').extract()
        availability = response.xpath('//div[@id="availability"]//text()').extract()
        items['product_name'] = ''.join(title).strip()
        items['product_sale_price'] = ''.join(sale_price).strip()
        items['product_category'] = ','.join(map(lambda x: x.strip(), category)).strip()
        items['product_availability'] = ''.join(availability).strip()
        yield items


Вот так уже можно запустить и получить необходимые результаты!

In [ ]:
!scrapy crawl AmazonProductSpider -o items.json #флаг -o - записать наш результат, запишем в виде jsonов (можно также в csv)

## Чего не хватает?

* Понимать бы, как вообще работают url-ссылки (что они из себя представлют)

* Хотим еще больше динамики для crawlingа - например, вводим запрос, получаем товары и ходим по ссылкам

Это все реализуем уже на семинаре на примера другого маркетплейса)

## Паук дня

![](https://live.staticflickr.com/4878/46966949422_871635f84f_b.jpg)

Это апулийский тарантул. Проживает в Италии, Испании и Португалии. Неядовитый (справедливости ради, подавляющее большинство пауков неядовитые, они могут укусить, но это бдет также, как и укус того же комара, неприятно, но нормально).

Один из самых известных тарантулов. В Средневековье считалось, что укус этого паука вызывает безумие (и вообще, считали, что он почти все смертельные болезни переносил с собой, но это не так).

Но что более интересно, что для того, чтобы не заболеть, итальянцы придумали противодействие: если укусили, то надо танцевать до упаду! А танец для того, чтобы не сойти с ума, назвали тарантелла (и он очень ассоциируется с Италией на данный момент)

Вот насколько находящееся вокруг нас влияет на культуру)